In [52]:
# Imports and constants
import time
import traceback
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import MetaData
from sqlalchemy import create_engine, select, insert, event
from sqlalchemy.sql import text
from sqlalchemy.engine import URL
import matplotlib.pyplot as plt

CHUNK_SIZE=10**6
OUT_PATH = 'output/eda_output.txt'
SEPARATOR = '-' * 100

GREEN = 'green'
YELLOW = 'yellow'
MIN_YEAR = 2020
MAX_YEAR = 2023

COLS = ['id', 
        'pickup_datetime', 'dropoff_datetime', 'ratecode_id', 
        'pu_location_id', 'do_location_id', 'passenger_count', 'trip_distance', 
        'fare_amount', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 
        'congestion_surcharge', 'improvement_surcharge', 'extra', 'payment_type']

# Functions

# def getODBCString():
#     SERVER = 'tcp:nyc-taxi-2024.database.windows.net,1433'
#     DATABASE = 'nyc_taxi_2024'
#     USERNAME = 'ishmakwana'
#     PASSWORD = 'xxx'

#     con_str = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
#     return URL.create("mssql+pyodbc", query={"odbc_connect": con_str})

# we want to print sql queries and their results to a output.txt instead of on the cell output window, and only log messages on the cell window. 
def pSep(num_lines=1):
    for l in range(num_lines):
        pOut(SEPARATOR)

def clOut():
    open(OUT_PATH, 'w').close()

def pOut(str, sep = 0):
    with open(OUT_PATH, 'a') as f:
        if sep > 0: pSep(sep)
        print(str, file=f)

def getSQLiteString():
    return 'sqlite:///db/taxi_db.db'

class TaxiDBReader:
    def __init__(self):
        self.md = MetaData()
        self.engn = create_engine(getSQLiteString())
        self.md.reflect(self.engn)
        self.year = MAX_YEAR
        self.taxi_type = YELLOW
        pOut('sql engine ready')

        with self.engn.connect() as conn:
            conn.rollback()

    def setTable(self, year=MAX_YEAR, taxi_type=YELLOW):
        self.year = year
        self.taxi_type = taxi_type

    def getTableName(self):
        return f'{self.taxi_type}_taxi_trips{self.year}'
    
clOut()
dr = TaxiDBReader()
        

In [53]:
# this will be repeated for other tables as well
dr.setTable(year=2023, taxi_type=YELLOW)
table_name = dr.getTableName()

In [54]:
# basic statistics
# note this will take long because we're trying to read the whole table
pOut('basic statistics', sep=1)
with dr.engn.connect() as conn:
    count = 1000
    # sql = text(f'SELECT * FROM {table_name} LIMIT {count}')
    sql = text(f'SELECT * FROM {table_name}')
    df = pd.read_sql(sql, conn)
    
    pOut(df.describe())
    with open(OUT_PATH, 'a') as f:
        df.info(buf=f)

    pOut('check for NaN values', sep=1)
    # count = 3000000
    # sql = text(f'SELECT * FROM {table_name}')
    # row_count = 0
    # for df in pd.read_sql(sql, conn, chunksize=count):
    #     row_count += len(df)
    rows_with_nan = df[df.isna().any(axis=1)]
    row_count = len(df)

    pOut(f'Rows with NaN: {len(rows_with_nan)}, rows processed: {row_count}') 

    pOut(f'total rows: {row_count}')

In [55]:
with dr.engn.connect() as conn:
    # start = 0
    # count = 10
    # sql = text(f'SELECT * FROM {table_name} LIMIT {count} OFFSET {start} ROWS FETCH NEXT {count} ROWS ONLY')
    # month = 1 # january
    # day = 1
    # hour = 1
    # sql = text(f'select * from {table_name} where strftime(\'%m\', pickup_datetime) = \'0{month}\'')
    
    # average passenger count by pickup location id
    pOut('average passenger count by pickup location id', sep=1)
    sql = text(f'select pu_location_id, avg(passenger_count) as avg_passenger_count from {table_name} where passenger_count > 0 group by pu_location_id')
    df = pd.read_sql(sql, conn)
    pOut(df)

    # number of trips by pickup location where no passengers where recorded
    pOut('number of trips by pickup location where no passengers where recorded')
    sql = text(f'select pu_location_id, count(passenger_count) as count_passenger_count from {table_name} where passenger_count = 0 group by pu_location_id')
    df = pd.read_sql(sql, conn)
    pOut(df)

    # maximum and minimum number of passengers
    pOut('maximum and minimum number of passengers')
    sql = text(f'select max(passenger_count) as max_passenger_count, min(passenger_count) as min_passenger_count from {table_name}')
    df = pd.read_sql(sql, conn)
    pOut(df)

    # total trips where passenger_count is 0
    pOut('total trips where passenger_count is 0')
    sql = text(f'select count(*) as trips_no_passenger from {table_name} where passenger_count=0')
    pOut(pd.read_sql(sql, conn))
    pOut(df)


In [ ]:
# fix zero trip
pOut('fix zero trip rows', sep=1)
with dr.engn.connect() as conn:

    # number of trips where passenger count is 0 and also trip_distance and total_amount
    sql = text(f'select count(*) as trips_zero_trip from {table_name} where trip_distance=0')
    pOut(pd.read_sql(sql, conn))
    sql = text(f'select count(*) as trips_zero_amount from {table_name} where total_amount=0')
    pOut(pd.read_sql(sql, conn))
    sql = text(f'select count(*) as trips_union from {table_name} where passenger_count=0 or trip_distance=0 or total_amount=0')
    pOut(pd.read_sql(sql, conn))
    sql = text(f'select count(*) as trips_intersection from {table_name} where passenger_count=0 and trip_distance=0 and total_amount=0')
    pOut(pd.read_sql(sql, conn))

    zero_td_sql = text(f'select distinct pu_location_id, do_location_id from {table_name} '
                          'WHERE trip_distance=0 ORDER BY pu_location_id, do_location_id ASC')
    # pOut(pd.read_sql(zero_td_sql, conn))
    avg_td_sql = text(f""" SELECT A.pu_location_id, A.do_location_id, AVG(A.trip_distance) 
                    FROM {table_name} as A WHERE A.trip_distance > 0 
                    GROUP BY A.pu_location_id, A.do_location_id
                    ORDER BY A.pu_location_id, A.do_location_id
                """)
    # pOut(pd.read_sql(avg_td_sql, conn))
    # add new column to store trip distance with fixed values
    add_ftd_sql = text(f"""
                ALTER TABLE {table_name} ADD COLUMN f_trip_distance FLOAT DEFAULT 0.0
                """)
    # conn.execute(add_ftd_sql)
    conn.commit()
    pOut('add new column ftd')

    check_ftd_sql = text(f"""
                SELECT AVG(f_trip_distance) as avg_ftd FROM {table_name}
                """)
    pOut(pd.read_sql(check_ftd_sql, conn))

    copy_existing_td_sql = text(f"""
                UPDATE {table_name} SET f_trip_distance=trip_distance
                """)
    # conn.execute(copy_existing_td_sql)
    conn.commit()
    pOut('copy td -> ftd')

    fix_td_sql = text(f"""
                WITH D AS (
                    SELECT pu_location_id as pu, do_location_id as do, AVG(trip_distance) as td
                    FROM {table_name} WHERE trip_distance > 0 GROUP BY pu_location_id, do_location_id
                ) 
                UPDATE {table_name} SET f_trip_distance=D.td FROM D 
                WHERE pu_location_id=D.pu 
                    AND do_location_id=D.do 
                    AND f_trip_distance=0
                """)
    # pOut(pd.read_sql(fix_td_sql, conn))
    # conn.execute(fix_td_sql)
    conn.commit()
    pOut('fix missing ftd')
    


In [ ]:
# possibility of fixing 'mta_tax', 'tip_amount', 'tolls_amount', but figure out a better filling than average filling
# & figure out a programmatic way to fix all the similar type of columns

In [ ]:
# fix total_amount by fare amount and other columns
pOut('fix total_amount by fare amount and other columns', sep=1)
# 'fare_amount', 'mta_tax', 'tip_amount', 'tolls_amount', 
# 'congestion_surcharge', 'improvement_surcharge', 'extra',

# add column f_fare_amount
# fix f_fare_amount using the same method used for trip_distance
# add column f_total_amount
# copy total_amount to fta
# fix fta by adding fare amount to other values (commented above)
with dr.engn.connect() as conn:
    add_ftd_sql = text(f"""
                ALTER TABLE {table_name} ADD COLUMN f_fare_amount FLOAT DEFAULT 0.0
                """)
    conn.execute(add_ftd_sql)
    conn.commit()
    pOut('add new column ffa')

    copy_existing_td_sql = text(f"""
                UPDATE {table_name} SET f_fare_amount=fare_amount
                """)
    conn.execute(copy_existing_td_sql)
    conn.commit()
    pOut('copy td -> ffa')

    fix_td_sql = text(f"""
                WITH D AS (
                    SELECT pu_location_id as pu, do_location_id as do, AVG(fare_amount) as fa
                    FROM {table_name} WHERE fare_amount > 0 GROUP BY pu_location_id, do_location_id
                ) 
                UPDATE {table_name} SET f_fare_amount=D.fa FROM D 
                WHERE pu_location_id=D.pu 
                    AND do_location_id=D.do 
                    AND f_fare_amount=0
                """)
    conn.execute(fix_td_sql)
    conn.commit()
    pOut('fix missing ffa')

    # F_TOTAL_AMOUNT
    add_ftd_sql = text(f"""
                ALTER TABLE {table_name} ADD COLUMN f_total_amount FLOAT DEFAULT 0.0
                """)
    conn.execute(add_ftd_sql)
    conn.commit()
    pOut('add new column fta')

    copy_existing_td_sql = text(f"""
                UPDATE {table_name} SET f_total_amount=total_amount
                """)
    conn.execute(copy_existing_td_sql)
    conn.commit()
    pOut('copy ta -> fta')

    fix_td_sql = text(f"""
                UPDATE {table_name} SET f_total_amount = (f_fare_amount + mta_tax + tip_amount + 
                                                        tolls_amount + congestion_surcharge + 
                                                        improvement_surcharge + extra)
                WHERE f_total_amount=0
                """)
    conn.execute(fix_td_sql)
    conn.commit()
    pOut('fix missing ffa')


In [ ]:
# fix passenger_count - set passenger count = 1 where it is 0 and...
# trip_distance != 0 and total_amount != 0
pOut('fix passenger_count - set passenger count = 1 where it is 0 and trip_distance != 0 and total_amount != 0', sep=1)
with dr.engn.connect() as conn:
    add_ftd_sql = text(f"""
                ALTER TABLE {table_name} ADD COLUMN f_passenger_count INT DEFAULT 0
                """)
    conn.execute(add_ftd_sql)
    conn.commit()
    pOut('add new column fpc')

    copy_existing_td_sql = text(f"""
                UPDATE {table_name} SET f_passenger_count=passenger_count
                """)
    conn.execute(copy_existing_td_sql)
    conn.commit()
    pOut('copy pc -> fpc')

    fix_td_sql = text(f"""
                UPDATE {table_name} SET f_passenger_count=1
                WHERE passenger_count=0 
                    AND f_trip_distance>0 
                    AND f_total_amount>0
                """)
    conn.execute(fix_td_sql)
    conn.commit()
    pOut('fix missing ftd')

In [ ]:
# drop missing passenger and 